In [2]:
%load_ext autoreload 
%autoreload 2
import torch
from wilds import get_dataset
from wilds.common.data_loaders import get_eval_loader
from wilds.datasets.unlabeled.wilds_unlabeled_dataset import WILDSUnlabeledSubset
import torchvision.transforms as transforms

from sklearn.metrics import roc_auc_score
import numpy as np

from paths import DATA_PATH, MODEL_PATH, create_predictions_path

from wilds_local.examples.transforms import initialize_transform
from gpu_utils import restrict_GPU_pytorch
restrict_GPU_pytorch('0')


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Using GPU:0


In [3]:
import pickle
algorithm = 'ERM'
split = "test"

# dataset = "camelyon17"
# model_name = 'densenet121'
dataset = "civilcomments"
model_name = "distilbert-base-uncased"

config = {
    "dataset": dataset,
    "algorithm": algorithm,
    "split": split,
    "model": model_name,
}

wilds_config = pickle.load(open('../../civilcomments_config', 'rb'))
transform = initialize_transform(wilds_config.transform, wilds_config, wilds_config.dataset, is_training=False)
# if config["dataset"] == "camelyon17":
#     transform = transforms.Compose(
#             [transforms.Resize(size=(96, 96),  max_size=None), 
#             transforms.ToTensor(),
#             transforms.Normalize(mean=[.485, .456, .406], std=[.229, .224, .225])])

if 'unlabeled' in config['split'] :
    full_unlabeled_dataset = get_dataset(
            dataset=config['dataset'],
            root_dir=DATA_PATH,
            unlabeled=True
        )
    
    all_idxs = list(range(len(full_unlabeled_dataset)))
    test_data = WILDSUnlabeledSubset(full_unlabeled_dataset, all_idxs, transform=transform)

else:
    full_dataset = get_dataset(dataset=dataset, root_dir=DATA_PATH)
    test_data = full_dataset.get_subset(split, transform=transform)

# Prepare the evaluation data loader
test_loader = get_eval_loader("standard", test_data, batch_size=32, num_workers=2)



In [4]:
import sys
from wilds_local.examples.models.initializer import initialize_model

# def initialize_torchvision_model(name, d_out, **kwargs):
#     import torchvision

#     # get constructor and last layer names
#     if name == 'wideresnet50':
#         constructor_name = 'wide_resnet50_2'
#         last_layer_name = 'fc'
#     elif name == 'densenet121':
#         constructor_name = name
#         last_layer_name = 'classifier'
#     elif name in ('resnet18', 'resnet34', 'resnet50', 'resnet101'):
#         constructor_name = name
#         last_layer_name = 'fc'
#     else:
#         raise ValueError(f'Torchvision model {name} not recognized')
#     # construct the default model, which has the default last layer
#     constructor = getattr(torchvision.models, constructor_name)
#     model = constructor(**kwargs)
#     # adjust the last layer
#     d_features = getattr(model, last_layer_name).in_features
#     if d_out is None:  # want to initialize a featurizer model
#         last_layer = Identity(d_features)
#         model.d_out = d_features
#     else: # want to initialize a classifier for a particular num_classes
#         last_layer = nn.Linear(d_features, d_out)
#         model.d_out = d_out
#     setattr(model, last_layer_name, last_layer)

#     return model

# def initialize_bert_based_model(config, d_out, featurize=False):
#     from wilds_local.examples.models.bert.bert import BertClassifier, BertFeaturizer
#     from wilds_local.examples.models.bert.distilbert import DistilBertClassifier, DistilBertFeaturizer

#     if config.pretrained_model_path:
#         print(f'Initialized model with pretrained weights from {config.pretrained_model_path}')
#         config.model_kwargs['state_dict'] = torch.load(config.pretrained_model_path, map_location=config.device)

#     if config.model == 'bert-base-uncased':
#         if featurize:
#             model = BertFeaturizer.from_pretrained(config.model, **config.model_kwargs)
#         else:
#             model = BertClassifier.from_pretrained(
#                 config.model,
#                 num_labels=d_out,
#                 **config.model_kwargs)
#     elif config.model == 'distilbert-base-uncased':
#         if featurize:
#             model = DistilBertFeaturizer.from_pretrained(config.model, **config.model_kwargs)
#         else:
#             model = DistilBertClassifier.from_pretrained(
#                 config.model,
#                 num_labels=d_out,
#                 **config.model_kwargs)
#     else:
#         raise ValueError(f'Model: {config.model} not recognized.')
#     return model

d_out = 2
if config["dataset"] == "ogbpcba":
    d_out = 128
model = initialize_model(wilds_config, d_out)

Some weights of DistilBertClassifier were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['pre_classifier.bias', 'classifier.weight', 'pre_classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [5]:
model_path = MODEL_PATH + '/' + config["dataset"] + '/' + config["algorithm"] + '/best_model.pth'
state_dict = torch.load(model_path)['algorithm']

if config["algorithm"] == "CORAL":
    remove_prefix = "model.0."
    state_dict = {k[len(remove_prefix):] if k.startswith(remove_prefix) else k: v for k, v in state_dict.items()}
    remove_prefix = "featurizer."
    state_dict = {k[len(remove_prefix):] if k.startswith(remove_prefix) else k: v for k, v in state_dict.items()}
else:
    remove_prefix = 'model.'
    state_dict = {k[len(remove_prefix):] if k.startswith(remove_prefix) else k: v for k, v in state_dict.items()}

model.load_state_dict(state_dict, strict=False)
model = model.cuda()


In [9]:
idx = 73529
# subset_idx = test_loader.dataset.indices[idx]
# test_loader.dataset.dataset[test_loader.dataset.indices[idx]]
test_loader.dataset[idx]

<class 'float'>
nan


(tensor([[  101,     1],
         [16660,     1],
         [  102,     1],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],
         [    0,     0],


In [8]:
from tqdm.notebook import tqdm
from scipy.special import softmax
model.eval()
# Takes about 6 minutes with a batch size of 16
all_y_true = []
all_y_pred = []
all_metadata = []
i = 0
for batch in tqdm(test_loader):
    if "unlabeled" in config['split']:
        x, metadata = batch
    else:
        x, y_true, metadata = batch
        
    y_pred = model(x.cuda())
    all_y_pred.append(y_pred.detach().cpu())
    all_metadata.append(metadata)

    if "unlabeled" not in config['split']:
        all_y_true.append(y_true)

    i += 1

all_y_pred = torch.concatenate(all_y_pred, axis=0)
all_y_pred = softmax(all_y_pred.detach().cpu().numpy(), axis=1)
all_metadata = torch.concatenate(all_metadata, axis=0)
if "unlabeled" not in config["split"]:
    all_y_true = torch.concatenate(all_y_true, axis=0)

  0%|          | 0/4181 [00:00<?, ?it/s]

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


<class 'str'><class 'str'>

you are going to be sent to a home for the stupid in 18 months if you keep spewing that stupdiity.OH yes - Were those evil Christian Missionaries (many of whom were slaughtered by the Indians, or lost their lives due to disease, starvation etc.) lumped in with those "genocidists" you mention ? (yes I did make up a word, just as you make up stories).

<class 'str'>
<class 'str'>You imply that Woody Allen is a child sex abuser so his view is not worth quoting. That's just a crude ad hominem.

He's considered a good candidate for a cyber-security post? He couldn't even keep emails from his paramour secure.
<class 'str'>
<class 'str'>Your comment is so racist!  How the heck did it make it past the civility test?

Lela, you admit no records exist to support your claims of a restrained religious hoard, but then state the atheists are worse? It's all bad, but it's all within human nature. If we can do away with any of the cults involved we might lessen future occur

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)



The woman is completely demonic. The democrat party is too terrified to cross her, because she has ordered the assassinations of dozens who have threatened her ambitions through the years. The woman is not only insane, she is pure evil.OMG can't you come up with anything original?!  What are you doing?!  Using the newspaper as your crib notes?!  Stop talking!  I'm embarrassed for you!!!  First you talk about googling nude women, then you can't even make an intelligent, unique statement!  You aren't very bright are you.....

<class 'str'><class 'str'>

Only one group was carrying torches.
Nazis and white supremacists.This idiot conveniently lumps a whole cast of characters into the "Religious Right", It is nothing other than a hit piece on Christians, and I don't care if he was a Bush adviser.

<class 'str'>
<class 'str'>Hilarious.
Almost as hilarious as every comment the thought Nazis disagree with here being flagged as abusive.
Carry on Dems, dig the hole deeper and deeper.
Just anot

KeyboardInterrupt: 

In [ ]:
import os
print(roc_auc_score(all_y_true.detach().cpu().numpy(), all_y_pred[:, 1]))
predictions_path = create_predictions_path(config)
print(predictions_path)
os.makedirs(predictions_path, exist_ok=True)
np.save(predictions_path + '/preds.npy', all_y_pred)
np.save(predictions_path + '/metadata.npy', all_metadata)
if "unlabeled" not in config["split"]:
    np.save(predictions_path + '/labels.npy', all_y_true)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/data/ddmg/frank/.conda/envs/frank/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/tmp/ipykernel_1970472/1553438322.py", line 2, in <module>
    print(roc_auc_score(all_y_true.detach().cpu().numpy(), all_y_pred[:, 1]))
AttributeError: 'list' object has no attribute 'detach'

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/data/ddmg/frank/.conda/envs/frank/lib/python3.9/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
  File "/data/ddmg/frank/.conda/envs/frank/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1435, in structured_traceback
    return FormattedTB.structured_traceback(
  File "/data/ddmg/frank/.conda/envs/frank/lib/python3.9/site-packages/IPython/core/ultratb.py", line 1326, in str